In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
data=pd.read_csv('/content/drive/MyDrive/project/train.csv')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# Define a custom function to combine columns
def combine_columns(row):
    # Convert both 'title' and 'text' to strings before concatenation
    return str(row['title']) + str(row['text'])

# Apply the custom function to create a new column 'Combine'
data['Combine'] = data.apply(combine_columns, axis=1)

# Drop the original columns
data.drop(['text', 'title', 'author'], axis=1, inplace=True)

# Reorder columns: 'id' first, 'Combine' next, 'label' last
cols = ['id', 'Combine'] + [col for col in data.columns if col not in ['id', 'Combine', 'label']] + ['label']
data = data[cols]

# View the first few rows
data.head()


,id,Combine,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,2,Why the Truth Might Get You FiredWhy the Truth...,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,1
4,4,Iranian woman jailed for fictional unpublished...,1


In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  # Use NLTK's list of stopwords
nltk.download('wordnet')  # Download the WordNet resource

# Importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Defining the object for stemming
porter_stemmer = PorterStemmer()

# Defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

# library that contains punctuation
string.punctuation

def remove_urls(text):
    if isinstance(text, str):  # Ensure the input is a string
        return re.sub(r'http\S+|www\S+', '', text, flags=re.MULTILINE)
    else:
        return text

def remove_stopwords(text):
    # Tokenize the text into words
    words = text.split()
    # Stem the words and remove stopwords
    text = ' '.join([porter_stemmer.stem(word) for word in words if word.lower() not in stop_words])
    # Lemmatize the words (optional, can choose to use either stemming or lemmatization)
    return ' '.join([wordnet_lemmatizer.lemmatize(word) for word in text.split()])

# Defining the function to remove punctuation and numbers
def clean_text(text):
    if not isinstance(text, str):  # Check if text is not a string
        return ''  # You can return an empty string or handle it however you want

    text = remove_urls(text)

    # Use regex to remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)  # This removes punctuation (anything that's not a word or whitespace)
    text = re.sub(r'\d+', '', text)  # This removes digits

    # Convert to lowercase
    text = text.lower()

    # Remove stopwords
    text = remove_stopwords(text)

    return text

# Storing the punctuation and number-free text
data['cl_text'] = data['Combine'].apply(lambda x: clean_text(x))

# Displaying the results
data[['Combine', 'cl_text']].head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Combine,cl_text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,hous dem aid didnt even see comey letter jason...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",flynn hillari clinton big woman campu breitbar...
2,Why the Truth Might Get You FiredWhy the Truth...,truth might get firedwhi truth might get fire ...
3,15 Civilians Killed In Single US Airstrike Hav...,civilian kill singl u airstrik identifiedvideo...
4,Iranian woman jailed for fictional unpublished...,iranian woman jail fiction unpublish stori wom...


In [ ]:
X=data['cl_text']
y=data['label']

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [ ]:
# Vectorize text data
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 4), analyzer='char')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# Define models
models = {
    'SVM': SVC(),
    #'Logistic Regression': LogisticRegression(solver='liblinear', multi_class='ovr'),
    #'Naive Bayes': MultinomialNB(),
    #'Decision Tree': DecisionTreeClassifier(random_state=0),
    #'KNN': KNeighborsClassifier(n_neighbors=3)
}


In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Train and evaluate models
for name, model in models.items():
    print(f"Training and evaluating {name}...")  # Add this line for clarity
    model.fit(X_train_tfidf, y_train)  # No need to encode y_train if labels are already numeric
    y_pred = model.predict(X_test_tfidf)

    # Get the unique predicted classes
    unique_pred_classes = np.unique(y_pred)
    # Use y_train directly since the labels are already numeric
    adjusted_target_names = [str(class_label) for class_label in unique_pred_classes]

    # Ensure the correct number of classes is used for classification_report
    print(f"\n{name} Classification Report:\n")
    print(classification_report(y_test, y_pred, labels=unique_pred_classes, target_names=adjusted_target_names))
    print("Accuracy:", accuracy_score(y_test, y_pred))


Training and evaluating SVM...

SVM Classification Report:

              precision    recall  f1-score   support

           0       0.96      0.95      0.95      3148
           1       0.95      0.95      0.95      3092

    accuracy                           0.95      6240
   macro avg       0.95      0.95      0.95      6240
weighted avg       0.95      0.95      0.95      6240

Accuracy: 0.9517628205128205
